In [ ]:
#Webscraper 18.10.2021

In [206]:
import requests # für die Connection
import bs4 as bs
from bs4 import BeautifulSoup
import pandas as pd
import os
from urllib.parse import urljoin

In [207]:
def get_soup(quote_page):
    page = requests.get(quote_page)
    soup = BeautifulSoup(page.text, 'html.parser') 
    return page, soup

In [2]:

## Download all relevant links from all archive pages
years = list(range(1996,2022))
months = ['01','02','03','04','05','06','07','08','09','10','11','12']

dfs = []
for year in years:
    print(year)
    for month in months:
        print(month)
        url = 'https://www.heise.de/newsticker/archiv/{}/{}/'.format(year, month)
        page, soup = get_soup(url)
        
        url_list, title_list, brand_list = [],[],[]
        for archiv in soup.find_all('a',{'class':'a-article-teaser__link archive__link'}, href=True):
            url_list.append(urljoin(url, archiv.attrs['href']))
            title_list.append(archiv.attrs['title'].strip())
    
            brand = archiv.select_one('.a-article-branding')
            if brand:
                brand_list.append(brand.text.strip())
            else:
                brand_list.append('newsticker')  
        
        df = pd.DataFrame([title_list,url_list,brand_list]).T
        df.columns = ["title", "url", "brand"]
        dfs.append(df)
out = pd.concat(dfs)
out.reset_index(inplace=True) #in order to get consecutive index column
out.drop('index',axis=1, inplace = True)

# create folder
os.makedirs('heise_archiv/')

# save urls
out.to_csv('heise_archiv/urls.csv',encoding='utf-8-sig')

1996
01
02
03
04
05
06
07
08
09
10
11
12
1997
01
02
03
04
05
06
07
08
09
10
11
12
1998
01
02
03
04
05
06
07
08
09
10
11
12
1999
01
02
03
04
05
06
07
08
09
10
11
12
2000
01
02
03
04
05
06
07
08
09
10
11
12
2001
01
02
03
04
05
06
07
08
09
10
11
12
2002
01
02
03
04
05
06
07
08
09
10
11
12
2003
01
02
03
04
05
06
07
08
09
10
11
12
2004
01
02
03
04
05
06
07
08
09
10
11
12
2005
01
02
03
04
05
06
07
08
09
10
11
12
2006
01
02
03
04
05
06
07
08
09
10
11
12
2007
01
02
03
04
05
06
07
08
09
10
11
12
2008
01
02
03
04
05
06
07
08
09
10
11
12
2009
01
02
03
04
05
06
07
08
09
10
11
12
2010
01
02
03
04
05
06
07
08
09
10
11
12
2011
01
02
03
04
05
06
07
08
09
10
11
12
2012
01
02
03
04
05
06
07
08
09
10
11
12
2013
01
02
03
04
05
06
07
08
09
10
11
12
2014
01
02
03
04
05
06
07
08
09
10
11
12
2015
01
02
03
04
05
06
07
08
09
10
11
12
2016
01
02
03
04
05
06
07
08
09
10
11
12
2017
01
02
03
04
05
06
07
08
09
10
11
12
2018
01
02
03
04
05
06
07
08
09
10
11
12
2019
01
02
03
04
05
06
07
08
09
10
11
12
2020
01
02
03
04

In [3]:
out

,title,url,brand
0,Surface: Microsoft baut eigenes Prozessorteam auf,https://www.heise.de/news/Surface-Microsoft-ba...,newsticker
1,Google Cloud: Künftig verstärkt auch im Rechen...,https://www.heise.de/news/Google-Cloud-Kuenfti...,iX Magazin
2,Tesla bietet bundesweit Ökostrom an,https://www.heise.de/news/Tesla-bietet-bundesw...,newsticker
3,Apple entfernt Bibel- und Koran-Apps aus chine...,https://www.heise.de/news/Apple-entfernt-Bibel...,Mac & i
4,iX-Workshop: KI hands-on – Machine Learning & ...,https://www.heise.de/news/iX-Workshop-KI-hands...,iX Magazin
...,...,...,...
237721,Überwachungskapitalismus: Shoshana Zuboff ford...,https://www.heise.de/news/Ueberwachungskapital...,newsticker
237722,US-Kaufhausriese Macy's zieht vor Gericht gege...,https://www.heise.de/news/US-Kaufhausriese-Mac...,newsticker
237723,"Freitag: Oracle-Itanium, Energie-Vergleich, Lä...",https://www.heise.de/news/Freitag-Oracle-Itani...,newsticker
237724,Itanium-Streit entschieden: Oracle muss HPE 3 ...,https://www.heise.de/news/Itanium-Streit-entsc...,newsticker


In [4]:
out.groupby('brand').count() #nur newsticker, MIT Technology Review und c't Magazin scrapen

,title,url
brand,,
Autos,1747,1747
BrandWorld,230,230
Developer,7460,7460
MIT Technology Review,6191,6191
Mac & i,9732,9732
Make,1844,1844
Security,11401,11401
Telepolis,1635,1635
c't Fotografie,1398,1398


In [21]:
out_selected_brands = out[out['brand'].isin(["newsticker","c't Magazin","MIT Technology Review"])].reset_index().drop('index',axis=1) #nur newsticker, 'c't und MIT Technology Review; z.B. Mac & i würde nur Apple überrepräsentieren

out_selected_brands.to_csv('heise_archiv/urls_selected_brands.csv',encoding='utf-8-sig')

In [461]:
import pandas as pd
out_selected_brands = pd.read_csv('heise_archiv/urls_selected_brands.csv',encoding='utf-8-sig', index_col = 'Unnamed: 0')

In [462]:
out_selected_brands

,title,url,brand
0,Surface: Microsoft baut eigenes Prozessorteam auf,https://www.heise.de/news/Surface-Microsoft-ba...,newsticker
1,Tesla bietet bundesweit Ökostrom an,https://www.heise.de/news/Tesla-bietet-bundesw...,newsticker
2,Gigantische Megakonstellation: Ruanda schlägt ...,https://www.heise.de/news/Gigantische-Megakons...,newsticker
3,"Kerkeling, Zeh, Fitzek und mehr gegen ""Zwangsl...",https://www.heise.de/news/Kerkeling-Zeh-Fitzek...,newsticker
4,Xbox Mini Fridge: Microsofts Meme-Kühlschrank ...,https://www.heise.de/news/Xbox-Mini-Fridge-Mic...,newsticker
...,...,...,...
193568,Überwachungskapitalismus: Shoshana Zuboff ford...,https://www.heise.de/news/Ueberwachungskapital...,newsticker
193569,US-Kaufhausriese Macy's zieht vor Gericht gege...,https://www.heise.de/news/US-Kaufhausriese-Mac...,newsticker
193570,"Freitag: Oracle-Itanium, Energie-Vergleich, Lä...",https://www.heise.de/news/Freitag-Oracle-Itani...,newsticker
193571,Itanium-Streit entschieden: Oracle muss HPE 3 ...,https://www.heise.de/news/Itanium-Streit-entsc...,newsticker


In [ ]:
path = 'C:/Users/Albina/JLUbox/DynTOBI/2021_aktuelle_Entwicklungen/Webscraper/' 
import pandas as pd
out_selected_brands = pd.read_csv(path + 'heise_archiv/urls_selected_brands.csv',encoding='utf-8-sig', index_col = 'Unnamed: 0')

import requests # für die Connection
import bs4 as bs
from bs4 import BeautifulSoup
import pandas as pd
import os
from urllib.parse import urljoin


def get_soup(quote_page):
    page = requests.get(quote_page)
    soup = BeautifulSoup(page.text, 'html.parser') 
    return page, soup


import time
import justext
import numpy as np
import os

frame = out_selected_brands.copy()
print(out_selected_brands.shape)
errorrate=0

#remove errors.txt file
if os.path.exists(path + 'heise_archiv/errors.txt'):
    os.remove(path + 'heise_archiv/errors.txt')
    print("The file has been deleted successfully")

for i,row in frame.iterrows():
    
    print(i)

    try:
        print(row.url)
        response,soup = get_soup(row.url)
        paragraphs = justext.justext(response.content, justext.get_stoplist("German"))
        b = [paragraph.text.replace('\n',' ') for paragraph in paragraphs if not paragraph.is_boilerplate]
        text = "".join(b)
        if text == '' : 
            print(i, row.url, 'no text')

        author = soup.find_all('meta',{'name':'author'})[0]['content']
        description = soup.find_all('meta',{'property':'og:description'})[0]['content']
        date = soup.find_all('meta',{'name':'date'})[0]['content']

        tmpdf = pd.DataFrame([text, author, description, date, row.title, row.url, row.brand])
        
        mode, header= ('w', True) if i <1 else ("a", False)
        tmpdf.T.to_csv(path + 'heise_archiv/heise_archiv.csv',encoding='utf-8', mode=mode, header=header)
        
    except Exception as e: 
        print('ERROR',e)
        print(i, row.url)
        errorrate+=1
        print('Current Errorrate:' + str(errorrate/i))
        with open(path + 'heise_archiv/' + 'errors.txt', 'a') as f: #stream urls in txt-file which couldn't be scrapped
            f.write(row.url)
            f.write('\n')

In [204]:
#Error txt-Datei einlesen und Inhalte nachscrapen
with open(path + 'heise_archiv/' + 'errors.txt', 'r') as f: #stream urls in txt-file which couldn't be scrapped
        error_list = []   
        for line in f:
            error_list.append(line.strip())
error_list[0:5]

['https://www.heise.de/news/Juengstes-CDU-Vorstandsmitglied-Klimapolitik-muss-Schwerpunkt-werden-6220473.html',
 'https://www.heise.de/news/Schwerin-Minister-mahnt-nach-Hackerattacke-zur-Vorsicht-weiter-Probleme-6220387.html',
 'https://www.heise.de/news/Nach-Dreh-auf-der-ISS-Russisches-Filmteam-zurueck-auf-der-Erde-6220393.html',
 'https://www.heise.de/news/Amazon-will-mehr-Abstimmung-mit-Behoerden-gegen-Produktpiraten-6220381.html',
 'https://www.heise.de/news/Facebook-schafft-fuer-virtuelle-Welt-Metaversum-10-000-Jobs-in-Europa-6220365.html']

In [292]:
#Errors resultieren aus fehlender Autorenangabe bzw. falschen Links oder heise-download links
# errors are mostly caused by missing author names
for i, url in enumerate(error_list):
    print(i)
    
    try:
        print(url)
        response,soup = get_soup(url)
        paragraphs = justext.justext(response.content, justext.get_stoplist("German"))
        b = [paragraph.text.replace('\n',' ') for paragraph in paragraphs if not paragraph.is_boilerplate]
        text = "".join(b)
        if text == '' : 
            print(i, url, 'no text')

        author = eval(soup.find('script',attrs= {'type':'application/ld+json'}).text.strip())['author']['name']
        description = soup.find_all('meta',{'property':'og:description'})[0]['content']
        date = soup.find_all('meta',{'name':'date'})[0]['content']

        tmpdf_errors = pd.DataFrame([text, author, description, date, url])
        mode, header= ('w', True) if i <1 else ("a", False)
        
        tmpdf_errors.T.to_csv(path + 'heise_archiv/heise_archiv_errors.csv',encoding='utf-8', mode=mode, header=header)
        
    except Exception as e: 
        print('ERROR',e)
        with open(path + 'heise_archiv/' + 'errors_errors.txt', 'a') as f: #stream urls in txt-file which couldn't be scrapped
            f.write(url)
            f.write('\n')

0
https://www.heise.de/news/Juengstes-CDU-Vorstandsmitglied-Klimapolitik-muss-Schwerpunkt-werden-6220473.html
1
https://www.heise.de/news/Schwerin-Minister-mahnt-nach-Hackerattacke-zur-Vorsicht-weiter-Probleme-6220387.html
2
https://www.heise.de/news/Nach-Dreh-auf-der-ISS-Russisches-Filmteam-zurueck-auf-der-Erde-6220393.html
3
https://www.heise.de/news/Amazon-will-mehr-Abstimmung-mit-Behoerden-gegen-Produktpiraten-6220381.html
4
https://www.heise.de/news/Facebook-schafft-fuer-virtuelle-Welt-Metaversum-10-000-Jobs-in-Europa-6220365.html
5
https://www.heise.de/news/Hackerangriffe-auf-mehrere-israelische-Krankenhaeuser-6220353.html
6
https://www.heise.de/news/Grossangelegter-Test-IT-Sicherheit-in-422-hessischen-Gemeinden-geprueft-6220304.html
7
https://www.heise.de/news/Studie-Lange-Weltraum-Aufenthalte-koennen-zu-Hirnschaeden-fuehren-6220300.html
8
https://www.heise.de/news/Face-Pay-Bezahlen-per-Gesichtserkennung-in-Moskauer-U-Bahn-moeglich-6220257.html
9
https://www.heise.de/news/Jugend

In [297]:
#error-file einlesen

heise_archiv_errors = pd.read_csv(path + 'heise_archiv\heise_archiv_errors.csv', index_col = 'Unnamed: 0')

In [298]:
heise_archiv_errors

,0,1,2,3,4
0,Das jüngste Vorstandsmitglied im CDU-Bundesvor...,dpa,"Wiebke Winter drängt die CDU, den Klimawandel ...",2021-10-18T09:36:00+02:00,https://www.heise.de/news/Juengstes-CDU-Vorsta...
0,Nach dem Hackerangriff auf kommunale IT-System...,dpa,Nachdem Hacker IT-Systeme in Schwerin und im b...,2021-10-18T07:05:00+02:00,https://www.heise.de/news/Schwerin-Minister-ma...
0,Nach Dreharbeiten auf der Internationalen Raum...,dpa,Ein russisches Filmteam ist nach Dreharbeiten ...,2021-10-18T06:51:00+02:00,https://www.heise.de/news/Nach-Dreh-auf-der-IS...
0,Amazon hat eine breitere Kooperation zwischen ...,dpa,Der US-Onlinehändler spricht sich für eine bre...,2021-10-18T06:42:00+02:00,https://www.heise.de/news/Amazon-will-mehr-Abs...
0,Facebook will in den kommenden fünf Jahren in ...,dpa,Facebook kommt nach Enthüllungen und massiven ...,2021-10-18T04:02:00+02:00,https://www.heise.de/news/Facebook-schafft-fue...
...,...,...,...,...,...
0,Die nach dem Startschuss für den digitalen Füh...,dpa,Die App für den digitalen Führerschein war nur...,2021-10-01T14:44:00+02:00,https://www.heise.de/news/Digitaler-Fuehrersch...
0,Die Menschen in Nigeria können künftig wohl wi...,dpa,Vier Monate nach Beginn einer Twitter-Sperre i...,2021-10-01T11:59:00+02:00,https://www.heise.de/news/Nigeria-Praesident-o...
0,"Um sogenannte ""Instagram-Gesichter"" zu verhind...",dpa,Botox-Behandlungen aus kosmetischen Gründen fü...,2021-10-01T09:22:00+02:00,https://www.heise.de/news/Instagram-Gesichter-...
0,Wie lange hält der Schutz der Corona-Impfungen...,"Alice Lanzke, dpa","Der Corona-Impfschutz lässt mit der Zeit nach,...",2021-10-01T08:50:00+02:00,https://www.heise.de/news/Corona-Impfungen-Wie...


In [300]:
heise_archiv_errors.columns = ['text','author','description','date','url'] #title und brand

In [305]:
import pandas as pd
out_selected_brands = pd.read_csv('heise_archiv/urls_selected_brands.csv',encoding='utf-8-sig', index_col = 'Unnamed: 0')

In [315]:
heise_archiv_errors_preprocessed = pd.merge(heise_archiv_errors,out_selected_brands, how = 'inner', on = 'url').drop_duplicates()

In [412]:
heise_archiv_errors_preprocessed

,text,author,description,date,url,title,brand,date_preprocessed
0,Das jüngste Vorstandsmitglied im CDU-Bundesvor...,dpa,"Wiebke Winter drängt die CDU, den Klimawandel ...",2021-10-18T09:36:00+02:00,https://www.heise.de/news/Juengstes-CDU-Vorsta...,Jüngstes CDU-Vorstandsmitglied: Klimapolitik m...,newsticker,2021-10-18
36,Nach dem Hackerangriff auf kommunale IT-System...,dpa,Nachdem Hacker IT-Systeme in Schwerin und im b...,2021-10-18T07:05:00+02:00,https://www.heise.de/news/Schwerin-Minister-ma...,Schwerin: Minister mahnt nach Hackerattacke zu...,newsticker,2021-10-18
72,Nach Dreharbeiten auf der Internationalen Raum...,dpa,Ein russisches Filmteam ist nach Dreharbeiten ...,2021-10-18T06:51:00+02:00,https://www.heise.de/news/Nach-Dreh-auf-der-IS...,Nach Dreh auf der ISS: Russisches Filmteam zur...,newsticker,2021-10-18
108,Amazon hat eine breitere Kooperation zwischen ...,dpa,Der US-Onlinehändler spricht sich für eine bre...,2021-10-18T06:42:00+02:00,https://www.heise.de/news/Amazon-will-mehr-Abs...,Amazon will mehr Abstimmung mit Behörden gegen...,newsticker,2021-10-18
144,Facebook will in den kommenden fünf Jahren in ...,dpa,Facebook kommt nach Enthüllungen und massiven ...,2021-10-18T04:02:00+02:00,https://www.heise.de/news/Facebook-schafft-fue...,"Facebook schafft für virtuelle Welt ""Metaversu...",newsticker,2021-10-18
...,...,...,...,...,...,...,...,...
5217,Wirtschaftsminister Peter Altmaier will auch a...,dpa,Die Mikroelektronik gilt als Schlüsseltechnolo...,2021-09-02T02:33:00+02:00,https://www.heise.de/news/Altmaier-will-durch-...,Altmaier will durch Förderprojekte mehr Chip-F...,newsticker,2021-09-02
5218,"Kranzwasserschöpfer werden eingesetzt, um auf ...",dpa,Eine heiße Quelle südlich der Azoren erzeugt e...,2021-09-01T07:19:00+02:00,https://www.heise.de/news/Azoren-Expedition-de...,"Azoren-Expedition der ""Meteor"": Forscher auf d...",newsticker,2021-09-01
5219,Nach dem Tod eines 13 Jahre alten Jungen bei e...,dpa,Nachdem in der Nähe von Mailand ein 13-jährige...,2021-09-01T07:13:00+02:00,https://www.heise.de/news/Italien-Nationale-Re...,Italien: Nationale Regeln nach tödlichem E-Sco...,newsticker,2021-09-01
5220,Bei der Jagd nach Steuerbetrügern setzt die St...,dpa,Im Südwesten können nun online und anonym Hinw...,2021-09-01T06:44:00+02:00,https://www.heise.de/news/Baden-Wuerttemberg-N...,Baden-Württemberg: Neue Meldeplattform für ano...,newsticker,2021-09-01


In [420]:
dates=[]
for i in range(0,len(heise_archiv_errors_preprocessed)):
    dates.append(pd.to_datetime(heise_archiv_errors_preprocessed.iloc[i].date.split('T')[0], format = '%Y-%m-%d'))
    
heise_archiv_errors_preprocessed['date_preprocessed'] = dates    

heise_archiv_errors_preprocessed.sort_values(by = 'date_preprocessed', inplace = True) #konkatenieren, #wie viele NAs?

In [421]:
heise_archiv_errors_preprocessed

,text,author,description,date,url,title,brand,date_preprocessed
2808,Bis zum Jahr 2002 will man bei Siemens Nixdorf...,Peter Averkamp,Bis zum Jahr 2002 will man bei Siemens Nixdorf...,1998-04-29T00:00:00,https://www.heise.de/newsticker/meldung/SNI-lo...,SNI löst sich von Mips,newsticker,1998-04-29
2812,Einen besonders heimtückischen Fehler in Excel...,Dieter Brors,Einen besonders heimtückischen Fehler in Excel...,1998-09-02T00:00:00,https://www.heise.de/newsticker/meldung/Heimtu...,Heimtückischer Fehler in Excel 97,newsticker,1998-09-02
2811,NaN,Kersten Auel,Nachdem das Schicksal der SGI-Tochter Cosmo So...,1998-09-14T00:00:00,https://www.heise.de/newsticker/meldung/Platin...,Platinum kauft Cosmo Software,newsticker,1998-09-14
2809,Internet gratis in UKSeit gestern bietet die b...,Axel Kossel,Seit gestern bietet die britische Dixons Group...,1998-09-23T00:00:00,https://www.heise.de/newsticker/meldung/Intern...,Internet gratis in UK,newsticker,1998-09-23
2810,"Bei einem ""Ready for change"" überschriebenen V...",nie,"Bei einem ""Ready for change"" überschriebenen V...",1998-09-23T00:00:00,https://www.heise.de/newsticker/meldung/Eine-M...,Eine Milliarde vernetzte PCs,newsticker,1998-09-23
...,...,...,...,...,...,...,...,...
72,Nach Dreharbeiten auf der Internationalen Raum...,dpa,Ein russisches Filmteam ist nach Dreharbeiten ...,2021-10-18T06:51:00+02:00,https://www.heise.de/news/Nach-Dreh-auf-der-IS...,Nach Dreh auf der ISS: Russisches Filmteam zur...,newsticker,2021-10-18
144,Facebook will in den kommenden fünf Jahren in ...,dpa,Facebook kommt nach Enthüllungen und massiven ...,2021-10-18T04:02:00+02:00,https://www.heise.de/news/Facebook-schafft-fue...,"Facebook schafft für virtuelle Welt ""Metaversu...",newsticker,2021-10-18
180,Mehrere israelische Krankenhäuser und medizini...,dpa,Eine Ransomware legt Krankenhaussysteme lahm. ...,2021-10-18T01:54:00+02:00,https://www.heise.de/news/Hackerangriffe-auf-m...,Hackerangriffe auf mehrere israelische Kranken...,newsticker,2021-10-18
108,Amazon hat eine breitere Kooperation zwischen ...,dpa,Der US-Onlinehändler spricht sich für eine bre...,2021-10-18T06:42:00+02:00,https://www.heise.de/news/Amazon-will-mehr-Abs...,Amazon will mehr Abstimmung mit Behörden gegen...,newsticker,2021-10-18


In [446]:
heise_archiv = pd.read_csv(path + 'heise_archiv\heise_archiv.csv', index_col = 'Unnamed: 0')

In [447]:
heise_archiv

,0,1,2,3,4,5,6
0,Eine Stellenausschreibung bei Microsoft bestät...,Mark Mantel,Surface-Tablets und -Notebooks könnten künftig...,2021-10-18T14:47:00+02:00,Surface: Microsoft baut eigenes Prozessorteam auf,https://www.heise.de/news/Surface-Microsoft-ba...,newsticker
0,Der Elektroautohersteller Tesla weitet seinen ...,Andreas Wilkens,Nun können nicht nur Menschen in Bayern und Ba...,2021-10-18T14:27:00+02:00,Tesla bietet bundesweit Ökostrom an,https://www.heise.de/news/Tesla-bietet-bundesw...,newsticker
0,Die erst wenige Monate alte Raumfahrtagentur R...,Martin Holland,Schon aktuelle Pläne für mehrere Zehntausend I...,2021-10-18T12:28:00+02:00,Gigantische Megakonstellation: Ruanda schlägt ...,https://www.heise.de/news/Gigantische-Megakons...,newsticker
0,"Autorinnen und Autoren, Verlage und der Buchha...",Andreas Wilkens,"Der Deutsche Bibliotheksverband meint, der App...",2021-10-18T11:57:00+02:00,"Kerkeling, Zeh, Fitzek und mehr gegen ""Zwangsl...",https://www.heise.de/news/Kerkeling-Zeh-Fitzek...,newsticker
0,"Er sieht aus wie eine Xbox Series X, ist aber ...",Daniel Herbig,Ab dem 19. Oktober kann man ihn endlich bestel...,2021-10-18T11:03:00+02:00,Xbox Mini Fridge: Microsofts Meme-Kühlschrank ...,https://www.heise.de/news/Xbox-Mini-Fridge-Mic...,newsticker
...,...,...,...,...,...,...,...
0,Insgesamt hat sich die Community der Internet-...,Stefan Krempl,Laut einer Social-Media-Analyse beklagten vers...,2021-10-01T07:15:00+02:00,Studie: Nur Verschwörungserzähler und Rechtsex...,https://www.heise.de/news/Studie-Nur-Verschwoe...,newsticker
0,"In der aktuellen ""entscheidenden Dekade der Di...",Stefan Krempl,Die US-Ökonomin hält das Einschreiten der Öffe...,2021-10-01T06:56:00+02:00,Überwachungskapitalismus: Shoshana Zuboff ford...,https://www.heise.de/news/Ueberwachungskapital...,newsticker
0,Eines der längsten Verfahren der IT-Branche is...,Frank Schräer,Oracle-Niederlage im Itanium-Streit + Energiek...,2021-10-01T06:30:00+02:00,"Freitag: Oracle-Itanium, Energie-Vergleich, Lä...",https://www.heise.de/news/Freitag-Oracle-Itani...,newsticker
0,Oracle muss wegen Vertragsbruchs mehr als drei...,Frank Schräer,Eines der längsten IT-Verfahren ist vorbei. Na...,2021-10-01T05:14:00+02:00,Itanium-Streit entschieden: Oracle muss HPE 3 ...,https://www.heise.de/news/Itanium-Streit-entsc...,newsticker


In [448]:
heise_archiv_preprocessed = heise_archiv.drop_duplicates()

In [449]:
heise_archiv_preprocessed.columns = ['text','author','description','date','title','url','brand']

In [450]:
heise_archiv_preprocessed

,text,author,description,date,title,url,brand
0,Eine Stellenausschreibung bei Microsoft bestät...,Mark Mantel,Surface-Tablets und -Notebooks könnten künftig...,2021-10-18T14:47:00+02:00,Surface: Microsoft baut eigenes Prozessorteam auf,https://www.heise.de/news/Surface-Microsoft-ba...,newsticker
0,Der Elektroautohersteller Tesla weitet seinen ...,Andreas Wilkens,Nun können nicht nur Menschen in Bayern und Ba...,2021-10-18T14:27:00+02:00,Tesla bietet bundesweit Ökostrom an,https://www.heise.de/news/Tesla-bietet-bundesw...,newsticker
0,Die erst wenige Monate alte Raumfahrtagentur R...,Martin Holland,Schon aktuelle Pläne für mehrere Zehntausend I...,2021-10-18T12:28:00+02:00,Gigantische Megakonstellation: Ruanda schlägt ...,https://www.heise.de/news/Gigantische-Megakons...,newsticker
0,"Autorinnen und Autoren, Verlage und der Buchha...",Andreas Wilkens,"Der Deutsche Bibliotheksverband meint, der App...",2021-10-18T11:57:00+02:00,"Kerkeling, Zeh, Fitzek und mehr gegen ""Zwangsl...",https://www.heise.de/news/Kerkeling-Zeh-Fitzek...,newsticker
0,"Er sieht aus wie eine Xbox Series X, ist aber ...",Daniel Herbig,Ab dem 19. Oktober kann man ihn endlich bestel...,2021-10-18T11:03:00+02:00,Xbox Mini Fridge: Microsofts Meme-Kühlschrank ...,https://www.heise.de/news/Xbox-Mini-Fridge-Mic...,newsticker
...,...,...,...,...,...,...,...
0,AMD blickt gespannt auf die Supercomputing-Kon...,Christof Windeck,Der erste Exaflops-Superrechner dürfte mit AMD...,2021-09-01T06:30:00+02:00,"Bit-Rauschen, der Prozessor-Podcast: Wettrenne...",https://www.heise.de/meinung/Bit-Rauschen-der-...,c't Magazin
0,Der Fahrdienstvermittler Lyft wird für die ab ...,Frank Schräer,Elektroautos werden für den Chauffeurdienst mi...,2021-09-01T03:59:00+02:00,Roboter-Taxis: Lyft setzt ab 2023 auf selbstfa...,https://www.heise.de/news/Roboter-Taxis-Lyft-s...,newsticker
0,US-Sanktionen gegen chinesische Firmen bedrohe...,Daniel AJ Sokolov,Nokia pausiert sein Engagement für die Standar...,2021-09-01T02:35:00+02:00,US-Sanktionen gefährden Netz-Standards,https://www.heise.de/news/US-Sanktionen-gefaeh...,newsticker
0,Compuserve war der erfolgreichste Onlinepionie...,Urs Mansmann,Vor dem Start des Internets in den 90er-Jahren...,2021-10-15T17:40:00+02:00,"Wie AOL, BTX und Compuserve Privatkunden onlin...",https://www.heise.de/hintergrund/Wie-AOL-BTX-u...,c't Magazin


In [452]:
dates=[]
for i in range(0,len(heise_archiv_preprocessed)):
    dates.append(pd.to_datetime(heise_archiv_preprocessed.iloc[i].date.split('T')[0], format = '%Y-%m-%d'))
    
heise_archiv_preprocessed['date_preprocessed'] = dates    

heise_archiv_preprocessed.sort_values(by = 'date_preprocessed', inplace = True) #konkatenieren, #wie viele NAs?

In [453]:
heise_archiv_preprocessed

,text,author,description,date,title,url,brand,date_preprocessed
0,NaN,Christian Persson,...,1996-04-17T00:00:00,Sicherheit: Probleme mit Java und JavaScript,https://www.heise.de/newsticker/meldung/Sicher...,newsticker,1996-04-17
0,Auf der Konferenz der European Oracle User Gro...,Jo Bager,Auf der Konferenz der European Oracle User Gro...,1996-04-17T00:00:00,Network Computer: Konkurrenz für den PC?,https://www.heise.de/newsticker/meldung/Networ...,newsticker,1996-04-17
0,NaN,Christian Persson,NaN,1996-04-17T00:00:00,Patente: IBM kassiert ab,https://www.heise.de/newsticker/meldung/Patent...,newsticker,1996-04-17
0,NaN,Christian Persson,http://www.heise.de/ct/96/05/020/,1996-04-17T00:00:00,Abmahnung: Microsoft und Gravenreuth einigen sich,https://www.heise.de/newsticker/meldung/Abmahn...,newsticker,1996-04-17
0,"Eine, laut Arizona Microchip nicht zu ""knacken...",Harald Ronge,"Eine, laut Arizona Microchip nicht zu ""knacken...",1996-04-18T00:00:00,Sicherheit: Unidirektionales Coding-Verfahren,https://www.heise.de/newsticker/meldung/Sicher...,newsticker,1996-04-18
...,...,...,...,...,...,...,...,...
0,Die erst wenige Monate alte Raumfahrtagentur R...,Martin Holland,Schon aktuelle Pläne für mehrere Zehntausend I...,2021-10-18T12:28:00+02:00,Gigantische Megakonstellation: Ruanda schlägt ...,https://www.heise.de/news/Gigantische-Megakons...,newsticker,2021-10-18
0,Der Elektroautohersteller Tesla weitet seinen ...,Andreas Wilkens,Nun können nicht nur Menschen in Bayern und Ba...,2021-10-18T14:27:00+02:00,Tesla bietet bundesweit Ökostrom an,https://www.heise.de/news/Tesla-bietet-bundesw...,newsticker,2021-10-18
0,Die Regierung der Metropole Paris hat den Kamp...,Ben Schwan,Eine Kombination aus Mikrofon und Kamera dient...,2021-10-18T09:00:00+02:00,Paris: Neue Sensoren sollen Lärm-Rowdies überf...,https://www.heise.de/hintergrund/Paris-Neue-Se...,MIT Technology Review,2021-10-18
0,Valve gestattet auf seiner PC-Spieleplattform ...,Daniel Herbig,"Valve möchte auf Steam keine Spiele sehen, die...",2021-10-18T10:05:00+02:00,Steam: Valve will keine NFT-Spiele anbieten,https://www.heise.de/news/Steam-Valve-will-kei...,newsticker,2021-10-18


In [459]:
df_heise_article = pd.concat([heise_archiv_preprocessed, heise_archiv_errors_preprocessed], ignore_index=True).sort_values(by = 'date_preprocessed').dropna(subset = ['text']) #NA Texte wurden gedroppt

In [475]:
df_heise_article.reset_index(drop = 'index', inplace = True)

In [476]:
df_heise_article

,text,author,description,date,title,url,brand,date_preprocessed
0,Auf der Konferenz der European Oracle User Gro...,Jo Bager,Auf der Konferenz der European Oracle User Gro...,1996-04-17T00:00:00,Network Computer: Konkurrenz für den PC?,https://www.heise.de/newsticker/meldung/Networ...,newsticker,1996-04-17
1,"Eine, laut Arizona Microchip nicht zu ""knacken...",Harald Ronge,"Eine, laut Arizona Microchip nicht zu ""knacken...",1996-04-18T00:00:00,Sicherheit: Unidirektionales Coding-Verfahren,https://www.heise.de/newsticker/meldung/Sicher...,newsticker,1996-04-18
2,Ein Minus von 740 Millionen Dollar hat Compute...,Frank Möcke,Ein Minus von 740 Millionen Dollar hat Compute...,1996-04-18T00:00:00,Apple: Drastische Maßnahmen nach hohen Verlusten,https://www.heise.de/newsticker/meldung/Apple-...,newsticker,1996-04-18
3,Das Bayerische Landeskriminalamt hat eine Hotl...,Christian Persson,Das Bayerische Landeskriminalamt hat eine Hotl...,1996-04-18T00:00:00,Kinderpornographie: Kripo sucht Hinweise,https://www.heise.de/newsticker/meldung/Kinder...,newsticker,1996-04-18
4,Zum Redaktionsschluß der c't 5/96 wollte Apple...,Stephan Ehrmann,Zum Redaktionsschluß der c't 5/96 wollte Apple...,1996-04-19T00:00:00,Apple: Hohe Preise sollen aus der Krise helfen,https://www.heise.de/newsticker/meldung/Apple-...,newsticker,1996-04-19
...,...,...,...,...,...,...,...,...
190717,Der Elektroautohersteller Tesla weitet seinen ...,Andreas Wilkens,Nun können nicht nur Menschen in Bayern und Ba...,2021-10-18T14:27:00+02:00,Tesla bietet bundesweit Ökostrom an,https://www.heise.de/news/Tesla-bietet-bundesw...,newsticker,2021-10-18
190718,Die Regierung der Metropole Paris hat den Kamp...,Ben Schwan,Eine Kombination aus Mikrofon und Kamera dient...,2021-10-18T09:00:00+02:00,Paris: Neue Sensoren sollen Lärm-Rowdies überf...,https://www.heise.de/hintergrund/Paris-Neue-Se...,MIT Technology Review,2021-10-18
190719,Valve gestattet auf seiner PC-Spieleplattform ...,Daniel Herbig,"Valve möchte auf Steam keine Spiele sehen, die...",2021-10-18T10:05:00+02:00,Steam: Valve will keine NFT-Spiele anbieten,https://www.heise.de/news/Steam-Valve-will-kei...,newsticker,2021-10-18
190720,Am 9. Oktober 2021 wurde die von einer KI verv...,Wolfgang Stieler,"Ahmed Elgammal erklärt im TR-Interview, wie di...",2021-10-18T10:00:00+02:00,Making of: Wie eine KI Beethovens 10. Sinfonie...,https://www.heise.de/hintergrund/Making-of-Wie...,MIT Technology Review,2021-10-18


In [477]:
df_heise_article.to_csv(path + 'heise_archiv/heise_article.csv',encoding='utf-8')
df_heise_article.to_pickle(path + 'heise_archiv/heise_article.pkl')